In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# unit meter, the square in front of camera view, the robot origin offset is set as 0.6m
source = np.array([[0, 0.5], [1, 0.5], [1, -0.5], [0, -0.5]], dtype=np.float64)
source = source.transpose()

# image pixel target
target = np.array([[14.32 , 140.71], [ 120.78, 95.5], [199.49 ,96.84], [302.7 ,140.71]], dtype=np.float64)

# transform to camera coordinate, not scaled yet
target = np.array([160 - target[:, 1], target[:, 0]])

# estimated scale factor for sensor

In [3]:
T = tf.constant(target)
S = tf.constant(source)

CameraOffset = tf.constant(0.6, dtype=tf.float64)
S = tf.stack([S[0, :] + CameraOffset, S[1, :]])

ScaleFactor = tf.constant(2000.0, dtype=tf.float64)
T = T / ScaleFactor

C = tf.Variable([0, 0.079999999999945434, 0.49999915892264074], dtype=tf.float64)
Cx, Cy, Cz = C[0], C[1], C[2]

E = tf.Variable([0.039999999999972717, 0.079999999999945434, -0.040000036913171277], dtype=tf.float64)
Ex, Ey, Ez = E[0], E[1], E[2]

Sx, Sy = S[0, :], S[1, :]
Tx, Ty = T[0, :], T[1, :]

lossX = -Ex*(Cx + Ez - Sx) + Ez*(Cz + Ex) + Tx*(Cx + Ez - Sx)
lossY = -Ey*(Cx + Ez - Sx) + Ez*(-Cy + Ey + Sy) + Ty*(Cx + Ez - Sx)

ForwardTx = (Cx*Ex - Cz*Ez - Ex*Sx)/(Cx + Ez - Sx)
ForwardTy = (Ey*(Cx + Ez - Sx) - Ez*(-Cy + Ey + Sy))/(Cx + Ez - Sx)

BackwardSx = (Cx*(Ex - Tx) - Ez*(Cz + Tx))/(Ex - Tx)
BackwardSy = (Cy*Ex + Cz*Ey - Cy*Tx + Ey*Tx - Cz*Ty - Ex*Ty)/(Ex - Tx)

#loss = tf.reduce_mean((BackwardSx - Sx) ** 2 +  (BackwardSy - Sy)**2)

#loss = tf.reduce_max([tf.abs(ForwardTx - Tx), tf.abs(ForwardTy - Ty)])
loss = tf.reduce_mean(lossX**2 + lossY**2)

learning_rate = tf.Variable(0.5)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train_op = optimizer.minimize(loss)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
#global train

for k in range(8):
    iter_range = 20000
    for i in range(iter_range):
        loss_val,  _ = sess.run([loss, train_op])
        if i == iter_range - 1:
            print('loss in iteration %d:' % k, loss_val)
    sess.run(learning_rate.assign(learning_rate / 2))

loss in iteration 0: 1.06140693681e-07
loss in iteration 1: 1.06140670578e-07
loss in iteration 2: 1.06140670578e-07
loss in iteration 3: 1.06140670578e-07
loss in iteration 4: 1.06140670578e-07
loss in iteration 5: 1.06140670578e-07
loss in iteration 6: 1.06140670578e-07
loss in iteration 7: 1.06140670578e-07


In [6]:
# validate forward inference
print(sess.run([ForwardTx, Tx]))
print(sess.run([ForwardTy, Ty]))
print(sess.run(loss))

[array([ 0.009645,  0.031915,  0.031915,  0.009645]), array([ 0.009645,  0.03225 ,  0.03158 ,  0.009645])]
[array([ 0.00715215,  0.06039214,  0.09974286,  0.15135785]), array([ 0.00716 ,  0.06039 ,  0.099745,  0.15135 ])]
1.06140670578e-07


In [11]:
sess.run([C, E])

[array([ 0.27883144,  0.07262362,  0.17211595]),
 array([ 0.04027264,  0.08037242, -0.05411853])]

In [8]:
print(sess.run([tf.reduce_sum(BackwardSx-Sx)**2]))
print(sess.run([tf.reduce_sum(BackwardSy-Sy)**2]))

[1.9592381471032017e-05]
[0.0016141816866117032]


In [7]:
print(sess.run([BackwardSx, Sx]))
print(sess.run([BackwardSy, Sy]))

[array([ 0.6       ,  1.6574276 ,  1.54699873,  0.6       ]), array([ 0.6,  1.6,  1.6,  0.6])]
[array([ 0.49994559,  0.52125865, -0.48108177, -0.49994559]), array([ 0.5,  0.5, -0.5, -0.5])]


In [12]:
sess.run(T)

array([[ 0.009645,  0.03225 ,  0.03158 ,  0.009645],
       [ 0.00716 ,  0.06039 ,  0.099745,  0.15135 ]])

In [15]:
sess.run(ScaleFactor)

2000.0000484463342

In [19]:
sess.run(CameraOffset)

0.58815510116706626

In [17]:
sess.run(ForwardTx * ScaleFactor)

array([ 18.10446478,  63.9103124 ,  63.9103124 ,  18.10446478])

In [18]:
sess.run(ForwardTy * ScaleFactor)

array([  14.99758888,  120.70440606,  199.56728247,  301.99749223])

In [20]:
sess.run(T* ScaleFactor)

array([[  19.29,   64.5 ,   63.16,   19.29],
       [  14.32,  120.78,  199.49,  302.7 ]])

In [36]:
target / 200

array([[ 0.09645,  0.3225 ,  0.3158 ,  0.09645],
       [ 0.0716 ,  0.6039 ,  0.99745,  1.5135 ]])

In [37]:
source

array([[ 0. ,  1. ,  1. ,  0. ],
       [ 0.5,  0.5, -0.5, -0.5]])

In [10]:
sess.run([ForwardTx, ForwardTy])

[array([ 0.009645,  0.031915,  0.031915,  0.009645]),
 array([ 0.00715215,  0.06039214,  0.09974286,  0.15135785])]

In [15]:
sess.run([Tx, Ty])

[array([ 0.009645,  0.03225 ,  0.03158 ,  0.009645]),
 array([ 0.00716 ,  0.06039 ,  0.099745,  0.15135 ])]

In [9]:
sess.run(CameraOffset)

0.59999999999999998

In [11]:
target

array([[  19.29,   64.5 ,   63.16,   19.29],
       [  14.32,  120.78,  199.49,  302.7 ]])